In [2]:
import json
import pickle
import logging
import requests
import warnings
from app import app
from keras import models
from tensorflow import keras
warnings.filterwarnings("ignore")
from flask.testing import FlaskClient
from flask import Flask, request, jsonify
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
with open("gradient_boost_model.pkl", "rb") as file:
    gradient_boost_model = pickle.load(file)
gradient_boost_model

GradientBoostingClassifier()

In [4]:
nn_model = keras.models.load_model("keras_model.h5")
nn_model

The '/' route returns a welcome message when you access the root URL.
The '/predict' route is designed to receive a POST request with a JSON payload containing features for prediction. It then uses the loaded machine learning models to make predictions based on the provided features.

Logging is crucial for monitoring and troubleshooting in a web application. It provides a record of events, such as received features and predictions, aiding in understanding the application's behavior and diagnosing issues effectively.

In [5]:
app = Flask("app")

# Configure logging
logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

@app.route('/', methods=['GET'])
def index():
    return 'Hello, welcome to Hotel booking platform'

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the request data from the user in JSON format
        request_json = request.get_json()

        #Define the input features
        features = {
            "lead_time": 30,
            "stays_in_week_nights": 3,
            "distribution_channel": "Online"
            }

        # Extract features from the request data
        features = request_json["features"]
        # Log the received features
        logging.info(f"Received features: {features}")

        # Make predictions using all three models
        features_array = np.array(features).reshape(1, -1)
        xgb_prediction = gradient_boost_model.predict(features_array)[0]
        nn_prediction = nn_model.predict(features_array)[0][0]

        # Log the prediction
        logging.info(f"Prediction: {prediction}")

        # Return the results as a JSON response
        return jsonify({
            "xgboost_prediction": int(xgb_prediction),
            "neural_network_prediction": float(nn_prediction)
        })

    except Exception as e:
        # Log the error
        logging.error(f"Error: {str(e)}")
        return jsonify({'error': str(e)})

if "app" == '_main_':
    app.run(port=5000, debug=True)

In [6]:
@app.route('/get_url', methods=['GET'])
def get_url():
    # Get the base URL of the application
    base_url = request.base_url

    return jsonify({"url": base_url})  

In [7]:
get_url

<function __main__.get_url()>

In [10]:
# Define individual features
lead_time = 30
stays_in_week_nights = 3
distribution_channel = "Online"

response = requests.post("http://127.0.0.1:5000/predict_gradient_boost", json={
    "features": {
        "lead_time": lead_time,
        "stays_in_week_nights": stays_in_week_nights,
        "distribution_channel": distribution_channel
    }
})

# Print the prediction results
print(response.json())

response = requests.post("http://127.0.0.1:5000/predict_neural_networks", json={
    "features": {
        "lead_time": lead_time,
        "stays_in_week_nights": stays_in_week_nights,
        "distribution_channel": distribution_channel
    }
})

# Print the prediction results
print(response.json())

{'error': "float() argument must be a string or a real number, not 'dict'"}
{'error': 'Failed to find data adapter that can handle input: (<class \'dict\'> containing {"<class \'str\'>"} keys and {"<class \'str\'>", "<class \'int\'>"} values), <class \'NoneType\'>'}


In [11]:
client = FlaskClient(app)
features = {
            "lead_time": 30,
            "stays_in_week_nights": 3,
            "distribution_channel": "Online"
            }

# Now, use the client to make requests:
response = client.post("/predict", json={"features": features})

In [12]:
 # Create a test client
client = FlaskClient(app)

# Specify the features individually
lead_time = 30
stays_in_week_nights = 3
distribution_channel = "Online"

# Make a POST request to the /predict endpoint
response = client.post("/predict", json={
    "lead_time": lead_time,
    "stays_in_week_nights": stays_in_week_nights,
    "distribution_channel": distribution_channel
})

# Access the response data
response_data = response.get_json()

# Print the prediction results
print(response_data)

{'error': "'features'"}
